In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import pprint
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from dataset import *
from runner import ExptRunner
from runner import ExptEvaluator
from networks.imageencoder import *
from networks.imagedecoder import *
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.dense import *
from networks.lossfunctions import *
from networks.multinet import *
from networks.special import *
from helpers import *
import utils
import simplereacherdimensions
import simplereacheradapters
# from env.simulator import simulator, evaluator

In [3]:
import warnings
warnings.filterwarnings("ignore", category=torch.serialization.SourceChangeWarning)

In [4]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [6]:
print (device)

cuda:2


In [7]:
if torch.cuda.is_available():
    print (torch.cuda.current_device())
    torch.cuda.set_device(device)
    print (torch.cuda.current_device())

0
2


In [8]:
root_folder = 'data/simple_reacher/training'
# number_of_trajectories = 10
number_of_trajectories = 5000

In [9]:
start = time.time()
train_data = SimpleReacherOnDemandDataset(root_folder, range(4750), device=device)
test_data = SimpleReacherOnDemandDataset(root_folder, range(4750, 5000), device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 5.406263113021851 sec


In [10]:
print (len(train_data))
print (len(test_data))
print (train_data.trajectory_data.device)
print (test_data.trajectory_data.device)

103274
5399
cuda:2
cuda:2


In [11]:
def run_expts(train_epochs):
    for i in range(len(expts)):
        arg_lists = expts[i]
        cons_args = arg_lists[0]
        train_args = {}
        if (len(arg_lists)) > 1:
            train_args = arg_lists[1]
        test_args = {}
        if (len(arg_lists)) > 2:
            test_args = arg_lists[2]
        
        runner = ExptRunner(train_data=train_data, test_data=test_data, device=device, **cons_args)
        print ("Experiment logs folder: {}".format(runner.log_folder))
        with open(runner.log_folder + '/expt.txt', 'w') as f:
            f.write(pprint.pformat(arg_lists))
        runner.train(train_epochs, **train_args)

In [12]:
# First number represents the number of layers, second : channels per each layer, third : kernel size
img_auto_encoder = lambda: ComposedAutoEncoder(x_dim=simplereacherdimensions.x_dim,
                                               img_res=simplereacherdimensions.img_res,
                                               layers_channels=[16, 16, 16, 16],
                                               useMaxPool=True,
                                               device=device)

img_auto_encoder_5_16_7 = lambda: ComposedAutoEncoder(x_dim=simplereacherdimensions.x_dim,
                                               img_res=simplereacherdimensions.img_res,
                                               layers_channels=[16, 16, 16, 16, 16],
                                               useMaxPool=True,
                                               device=device,
                                               kernel_size=7)

img_auto_encoder_5_32_7 = lambda: ComposedAutoEncoder(x_dim=simplereacherdimensions.x_dim,
                                               img_res=simplereacherdimensions.img_res,
                                               layers_channels=[32, 32, 32, 32, 32],
                                               useMaxPool=True,
                                               device=device,
                                               kernel_size=7)
img_auto_encoder_5_16_5 = lambda: ComposedAutoEncoder(x_dim=simplereacherdimensions.x_dim,
                                               img_res=simplereacherdimensions.img_res,
                                               layers_channels=[16, 16, 16, 16, 16],
                                               useMaxPool=True,
                                               device=device,
                                               kernel_size=5)

img_auto_encoder_5_32_5 = lambda: ComposedAutoEncoder(x_dim=simplereacherdimensions.x_dim,
                                               img_res=simplereacherdimensions.img_res,
                                               layers_channels=[32, 32, 32, 32, 32],
                                               useMaxPool=True,
                                               device=device,
                                               kernel_size=5)

In [14]:
expts = [
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder4-MSE',
        "net_func": img_auto_encoder,
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": mse_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_7-L1',
        "net_func": img_auto_encoder_5_16_7,
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": l1_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_7-L1',
        "net_func": img_auto_encoder_5_32_7,
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": l1_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_5-L1',
        "net_func": img_auto_encoder_5_16_5,
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": l1_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_5-L1',
        "net_func": img_auto_encoder_5_32_5,
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": l1_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
]

In [15]:
train_epochs = 10
run_expts(train_epochs)

Experiment logs folder: runs/08-10-20-54-33-ImageAutoEncoder-Reacher-ComposedAutoEncoder4-MSE
Experiment logs folder: runs/08-10-21-23-45-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_7-L1
Experiment logs folder: runs/08-10-22-03-54-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_7-L1
Experiment logs folder: runs/08-10-23-00-44-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_5-L1
Experiment logs folder: runs/08-10-23-31-30-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_5-L1


In [11]:
testchkpt = torch.load('runs/07-29-04-24-51-PolicyNoScalingOfInputsAndOutputs-Reacher-Dense-MSE/train_checkpoint.tar', map_location=device)
testNet = testchkpt['model']
testNet.eval()


Dense(
  (net): Sequential(
    (pol_fc0): Linear(in_features=6, out_features=32, bias=True)
    (pol_relu0): ReLU()
    (pol_fc1): Linear(in_features=32, out_features=32, bias=True)
    (pol_relu1): ReLU()
    (pol_fc2): Linear(in_features=32, out_features=3, bias=True)
    (pol_sig2): Sigmoid()
  )
)

In [14]:
chkpt_file = 'runs/07-19/07-20-00-08-11-ImageAutoEncoder-Reacher-ComposedAutoEncoder-L1/train_checkpoint.tar'

In [16]:
e = ExptEvaluator(chkpt_file, train_data, test_data, device)

NameError: name 'chkpt_file' is not defined

In [26]:
e.net = e.net.to(device)
print (e.net.device)

cuda:2


In [17]:
start = time.time()

for chkpt_file in utils.enumerate_files(rootdir='runs/08-10'):
    e = ExptEvaluator(chkpt_file, train_data, test_data, device)
    e.test()
    print ('Done with checkpoint file {}'.format(chkpt_file))

print ("Time Taken: {} sec".format(time.time() - start))

Done with checkpoint file runs/08-10/08-10-20-54-33-ImageAutoEncoder-Reacher-ComposedAutoEncoder4-MSE/train_checkpoint.tar
Done with checkpoint file runs/08-10/08-10-21-23-45-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_7-L1/train_checkpoint.tar
Done with checkpoint file runs/08-10/08-10-22-03-54-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_7-L1/train_checkpoint.tar
Done with checkpoint file runs/08-10/08-10-23-00-44-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_16_5-L1/train_checkpoint.tar
Done with checkpoint file runs/08-10/08-10-23-31-30-ImageAutoEncoder-Reacher-ComposedAutoEncoder_5_32_5-L1/train_checkpoint.tar
Time Taken: 728.2372713088989 sec


In [26]:
def save_sample(ip_batch, op_batch, filePath):
    cmap = plt.get_cmap('gray')
    ip_batch = ip_batch.cpu()
    op_batch = op_batch.cpu()
    fig = plt.figure()
    ax = plt.subplot(1, 2, 1)
    plt.imshow(ip_batch.reshape(256, 256), cmap=cmap, vmin=0, vmax=1)
    ax.title.set_text('Original')
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax = plt.subplot(1, 2, 2)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.imshow(op_batch.reshape(256, 256), cmap=cmap, vmin=0, vmax=1)
    ax.title.set_text('Recontructed')
    fig.savefig(filePath)
    plt.close()

In [27]:
start = time.time()

for chkpt_file in utils.enumerate_files(rootdir='runs/08-10'):
    chkpt = torch.load(chkpt_file, map_location=device)
    net = chkpt['model']
    data_adapter_func = chkpt['input_adapter']
    # data_to_label_adapter = chkpt['label_adapter']
    log_folder = os.path.dirname(chkpt_file)
    for dataset, setName in zip([train_data, test_data], ['training_set', 'test_set']):
        for crit in ['good', 'bad']:
            with open(os.path.join(log_folder, '{}_{}_samples.txt'.format(setName, crit)), 'r') as f:
                while True:
                    line = f.readline()
                    if len(line) == 0:
                        break
                    idx = int(line)
                    ip_batch = data_adapter_func(dataset[idx])
                    with torch.no_grad():
                        op_batch = net(ip_batch)
                        save_sample(ip_batch, op_batch, os.path.join(log_folder, '{}_{}_{}.png'.format(setName, crit, idx)))

print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 7.420027256011963 sec


In [42]:
class AdapterNet(nn.Module):
    def __init__(self, testNet, ip_adapter=None, op_adapter=None):
        super().__init__()
        self.testNet = testNet
        self.ip_adapter = ip_adapter if ip_adapter else lambda x:x
        self.op_adapter = op_adapter if op_adapter else lambda x:x
        
    def forward(self, data):
        ip = self.ip_adapter(data)
        op = self.testNet(ip)
        return self.op_adapter(op)

In [91]:
t475 = test_data.trajectory_data[test_data.trajectory_index[:, 0] == 475, :]

In [92]:
t475.shape

torch.Size([19, 9])

In [99]:
ip_adapter = lambda x:x.float()
op_adapter = lambda x:((x * (2 * simplereacherdimensions.joint_max)) - (simplereacherdimensions.joint_max))
ev = evaluator(AdapterNet(testNet, ip_adapter=ip_adapter, op_adapter=op_adapter))

In [100]:
t475_output = ev.rollout(t475)

Creating env


In [101]:
t475_output

tensor([[2.0023, 0.4158, 1.8592, 0.9967, 1.1594, 2.5409],
        [1.9043, 0.5139, 1.7611, 0.9996, 1.1593, 2.5384],
        [1.8053, 0.5956, 1.6621, 1.0007, 1.1592, 2.5375],
        [1.7063, 0.5854, 1.7592, 1.0011, 1.1592, 2.5371],
        [1.6073, 0.6034, 1.8582, 1.0012, 1.1592, 2.5370],
        [1.5083, 0.6397, 1.9572, 1.0013, 1.1592, 2.5369],
        [1.4093, 0.7053, 2.0562, 1.0013, 1.1592, 2.5369],
        [1.3102, 0.7835, 2.1552, 1.0013, 1.1592, 2.5369],
        [1.2112, 0.8817, 2.2542, 1.0013, 1.1592, 2.5369],
        [1.1122, 0.9807, 2.2044, 1.0013, 1.1592, 2.5369],
        [1.0132, 1.0797, 2.2021, 1.0013, 1.1592, 2.5369],
        [0.9142, 1.1061, 2.3001, 1.0013, 1.1592, 2.5369],
        [0.8152, 1.0081, 2.3929, 1.0013, 1.1592, 2.5369],
        [0.7162, 0.9091, 2.3035, 1.0013, 1.1592, 2.5369],
        [0.6172, 0.8319, 2.2383, 1.0013, 1.1592, 2.5369],
        [0.5182, 0.7456, 2.1949, 1.0013, 1.1592, 2.5369],
        [0.4192, 0.6712, 2.1471, 1.0013, 1.1592, 2.5369],
        [0.320

In [96]:
t475[:, :6]

tensor([[2.0023, 0.4158, 1.8592, 0.9967, 1.1594, 2.5409],
        [1.9702, 0.3573, 1.9128, 0.9967, 1.1594, 2.5409],
        [1.9658, 0.3536, 1.9160, 0.9967, 1.1594, 2.5409],
        [1.9073, 0.4041, 1.9532, 0.9967, 1.1594, 2.5409],
        [1.8453, 0.4557, 1.9929, 0.9967, 1.1594, 2.5409],
        [1.7844, 0.5060, 2.0321, 0.9967, 1.1594, 2.5409],
        [1.7239, 0.5562, 2.0713, 0.9967, 1.1594, 2.5409],
        [1.6633, 0.6065, 2.1104, 0.9967, 1.1594, 2.5409],
        [1.6027, 0.6567, 2.1495, 0.9967, 1.1594, 2.5409],
        [1.5421, 0.7070, 2.1887, 0.9967, 1.1594, 2.5409],
        [1.4815, 0.7573, 2.2278, 0.9967, 1.1594, 2.5409],
        [1.4209, 0.8075, 2.2670, 0.9967, 1.1594, 2.5409],
        [1.3603, 0.8578, 2.3061, 0.9967, 1.1594, 2.5409],
        [1.2997, 0.9080, 2.3452, 0.9967, 1.1594, 2.5409],
        [1.2391, 0.9583, 2.3844, 0.9967, 1.1594, 2.5409],
        [1.1785, 1.0086, 2.4235, 0.9967, 1.1594, 2.5409],
        [1.1179, 1.0588, 2.4627, 0.9967, 1.1594, 2.5409],
        [1.057

In [36]:
print (F.l1_loss(t475[:, :3], t475_output[:, :3]))
print (F.l1_loss(t475[-1:, :3], t475_output[-1:, :3]))

NameError: name 't475' is not defined

In [41]:
train_samples = np.random.randint(len(train_data), size=5)
test_samples = np.random.randint(len(test_data), size=5)

In [45]:
start = time.time()
cmap = plt.get_cmap('gray')

for chkpt_file in utils.enumerate_files(rootdir='runs/07-19'):
    log_folder = os.path.dirname(chkpt_file)
    chkpt = torch.load(chkpt_file, map_location=device)
    testNet = chkpt['model']
    testNet.eval()
    for idx in test_samples:
        fileName = 'TestSample_{}'.format(str(idx))
        fig = plt.figure()
        ip_batch = simplereacheradapters.It_scaled_adapter(test_data[idx.item()])
        with torch.no_grad():
            op_batch = testNet(ip_batch)
        ip_batch = ip_batch.cpu()
        op_batch = op_batch.cpu()
        plt.figure()
        ax = plt.subplot(1, 2, 1)
        plt.imshow(ip_batch.reshape(256, 256), cmap=cmap)
        ax.title.set_text('Original')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax = plt.subplot(1, 2, 2)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        plt.imshow(op_batch.reshape(256, 256), cmap=cmap)
        ax.title.set_text('Recontructed')
        plt.show()
        #fig.savefig(log_folder + '/' + filename + '.png')


print ("Time Taken: {} sec".format(time.time() - start))



AssertionError: 
The NVIDIA driver on your system is too old (found version 10010).
Please update your GPU driver by downloading and installing a new
version from the URL: http://www.nvidia.com/Download/index.aspx
Alternatively, go to: https://pytorch.org to install
a PyTorch version that has been compiled with your version
of the CUDA driver.

<Figure size 432x288 with 0 Axes>